In [46]:
import requests
import json
from pathlib import Path
import os
import sys
import pandas as pd
from datetime import datetime
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

import utils as util

In [47]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo" 
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [48]:
requestData = {
 
    # SiteName takes in two different parameters: [CameraOne, CameraTwo]
    # SceneName takes in different parameteres depending on SiteName:  [CameraOne [HospitalView]], [CameraTwo [SmallStreetSlide, FieldSlide, SchoolView]]

    # If you want to return all availalbe scans and images, comment out the Coordinates line.
   
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2022-01-01T00:00:00.000Z',
        'Maximum': '2023-02-27T15:43:16.000Z'
    },
    #'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  # comment out if you want to return all images
}

In [49]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [50]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [35]:
channelInfoData = channelResponse.json()
data = response.json()

In [51]:
# obtain measurement labels
skip_rate = 2
measurement_labels = []
for item in channelInfoData:
    for index, (key, value) in enumerate(item.items()):
        if index % skip_rate == 0:
            measurement_labels.append(value)

# map labels with measurements and drop redundant columns
df = pd.DataFrame(data)
count = 0
for i in measurement_labels:
    df[i] =df['Values'].apply(lambda x: x[count] )
    count = count + 1
df = df.drop(columns=['Values'])

In [52]:
df

,Timestamp,ImageUrls,ACTUAL_TEMPERATURE,HUMIDITY,ILLUMINATION,RAIN_COUNTER,SUNSHINEDURATION,WIND_SPEED,WIND_SPEED_CALC,Rain.Day.Counter,Rain.Diff,Sunshine.Day.Counter,Sunshine.Diff
0,2022-10-19T15:01:40.509111+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,13.4,88,0,277.600006,12023,16.299999,4.527778,14.2,0,1040,0
1,2022-10-19T17:08:44.80808+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,13.5,86,0,277.600006,12023,8.1,2.25,14.2,0,1040,0
2,2022-10-20T08:08:45.282687+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.299999,68,100000,277.600006,12346,21.5,5.972222,0,0,179,2
3,2022-10-20T13:08:44.972007+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,16.799999,78,4228,277.600006,12647,7.8,2.166667,0,0,488,3
4,2022-10-20T17:08:44.9777+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,13.5,92,0,277.600006,12669,0,0,0,0,510,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,2023-01-28T08:08:46.33017+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.6,44,16180,773.200012,5921,25.700001,7.138889,0,0,130,2
264,2023-01-28T13:08:44.747983+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,20.5,54,8380,773.200012,6221,23.4,6.5,0,0,430,3
265,2023-01-29T08:08:45.861822+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.299999,58,7448,773.200012,6524,16.9,4.694445,0,0,128,2
266,2023-01-29T13:08:45.031074+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.6,54,3942,773.200012,6825,13.3,3.694444,0,0,429,2


# Put it into function 
We wrap the data request into a nice reusable function and store it under the src folder and cann simply call it in the next step.

In [55]:
df = util.return_data()
df

,Timestamp,ImageUrls,ACTUAL_TEMPERATURE,HUMIDITY,ILLUMINATION,RAIN_COUNTER,SUNSHINEDURATION,WIND_SPEED,WIND_SPEED_CALC,Rain.Day.Counter,Rain.Diff,Sunshine.Day.Counter,Sunshine.Diff
0,2022-10-19T15:01:40.509111+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,13.4,88,0,277.600006,12023,16.299999,4.527778,14.2,0,1040,0
1,2022-10-19T17:08:44.80808+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,13.5,86,0,277.600006,12023,8.1,2.25,14.2,0,1040,0
2,2022-10-20T08:08:45.282687+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.299999,68,100000,277.600006,12346,21.5,5.972222,0,0,179,2
3,2022-10-20T13:08:44.972007+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,16.799999,78,4228,277.600006,12647,7.8,2.166667,0,0,488,3
4,2022-10-20T17:08:44.9777+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,13.5,92,0,277.600006,12669,0,0,0,0,510,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,2023-01-28T08:08:46.33017+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.6,44,16180,773.200012,5921,25.700001,7.138889,0,0,130,2
264,2023-01-28T13:08:44.747983+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,20.5,54,8380,773.200012,6221,23.4,6.5,0,0,430,3
265,2023-01-29T08:08:45.861822+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.299999,58,7448,773.200012,6524,16.9,4.694445,0,0,128,2
266,2023-01-29T13:08:45.031074+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.6,54,3942,773.200012,6825,13.3,3.694444,0,0,429,2


# Questions
* How to return all images if timestamp is not known  --> like an "all" function...